In [15]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
pgpass file created at C:\Users\JEAUL2\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [34]:
import os
import pandas as pd
import wrds
from datetime import datetime
from dateutil.relativedelta import relativedelta




date = datetime(1980, 1, 2)
os.makedirs("constituants_mkt", exist_ok=True)

all_stock = set()
while date < datetime(2024, 1, 2):
  date_str = date.strftime("%Y-%m-%d")
  print(date_str)
  q = f"""
      select a.permno, a.date, a.prc, a.shrout,
        (abs(a.prc) * a.shrout * 1000) as market_cap,
        b.ticker, b.comnam
      from crsp.dsf a
      join crsp.stocknames b
        on a.permno = b.permno
      and a.date between b.namedt and b.nameenddt
      where a.date = (
          select max(date) from crsp.dsf
          where date <= '{date_str}'
      )
        and a.prc > 0
        and a.shrout > 0
      order by market_cap desc
      limit 500;

  """
  mrk_df = db.raw_sql(q)['permno']
  mrk_csv = os.path.join('constituants_mkt', f"{date.year}.csv")
  mrk_df.to_csv(mrk_csv, index=False)
  
  stocks =  mrk_df.tolist()
  all_stock.update(stocks)

  date += relativedelta(years=1)


1980-01-02
1981-01-02
1982-01-02
1983-01-02
1984-01-02
1985-01-02
1986-01-02
1987-01-02
1988-01-02
1989-01-02
1990-01-02
1991-01-02
1992-01-02
1993-01-02
1994-01-02
1995-01-02
1996-01-02
1997-01-02
1998-01-02
1999-01-02
2000-01-02
2001-01-02
2002-01-02
2003-01-02
2004-01-02
2005-01-02
2006-01-02
2007-01-02
2008-01-02
2009-01-02
2010-01-02
2011-01-02
2012-01-02
2013-01-02
2014-01-02
2015-01-02
2016-01-02
2017-01-02
2018-01-02
2019-01-02
2020-01-02
2021-01-02
2022-01-02
2023-01-02


In [35]:
print(len(all_stock))


2126


<class 'set'>
<class 'list'>


### Le code plus haut extrait les 500 plus grosses market cap par années

In [37]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import wrds

# ------------------------------------------------------------
# Paramètres à adapter
# ------------------------------------------------------------
PERMNOS     = list(all_stock) # <-- ta liste de PERMNO (exemples)
START_DATE  = "1976-01-01"
END_DATE    = "2025-01-01"
OUTPUT_DIR  = "./wrds_exports"


os.makedirs(OUTPUT_DIR, exist_ok=True)



# ------------------------------------------------------------
# 1) CRSP daily returns (RET) — ajustés splits & dividendes
# ------------------------------------------------------------
def get_crsp_returns(db, permnos, start, end):
    """
    Récupère les rendements CRSP daily (RET) pour une liste de PERMNO.
    RET inclut les distributions (dividendes) et les splits (ajustements CRSP).
    Option : inclure les delisting returns (DLRET) sur le dernier jour.
    """
    # Base: crsp.dsf
    sql = f"""
        select date, permno, ret
        from crsp.dsf
        where permno in ({",".join(str(int(x)) for x in permnos)})
          and date between '{start}' and '{end}'
        order by date, permno
    """
    df = db.raw_sql(sql, date_cols=["date"])

    # Nettoyage RET -> float
    df["ret"] = pd.to_numeric(df["ret"], errors="coerce")  # coerce '.', 'B', 'C' éventuels en NaN

    

    return df  # colonnes : date, permno, ret


def get_crsp_prices(db, permnos, start, end, adjust="split"):
    """
    Prix quotidiens CRSP pour une liste de PERMNO.
    - Columns toujours renvoyées: date, permno, prc
    - Si adjust="split": ajoute adjprc = |prc| * cfacpr (ajusté des splits/stock dividends)
    - include_delisting_price:
        * ajoute une ligne à dlstdt si absente, avec prc = prev_prc * (1 + dlret)
        * utile pour avoir une 'dernière cotation' cohérente le jour de radiation
    Notes:
      - prc est mis en valeur absolue (CRSP signe parfois le prix).
      - adjprc n'ajuste PAS les dividendes cash (pour le total return, il faut cumuler les RET).
    """
    import numpy as np
    import pandas as pd

    # 1) Récup prix + facteurs d'ajustement (cfacpr)
    sql = f"""
        select date, permno, prc, cfacpr
        from crsp.dsf
        where permno in ({",".join(str(int(x)) for x in permnos)})
          and date between '{start}' and '{end}'
        order by date, permno
    """
    px = db.raw_sql(sql, date_cols=["date"])
    # Nettoyage numérique
    px["prc"] = pd.to_numeric(px["prc"], errors="coerce").abs()
    px["cfacpr"] = pd.to_numeric(px["cfacpr"], errors="coerce")

    
    # 3) Ajustement split (adjprc)
    if adjust == "split":
        # cfacpr peut être NaN sur certaines dates très anciennes: mettre 1.0 par défaut
        px["adjprc"] = px["prc"] * px["cfacpr"].fillna(1.0)

    # 4) Colonnes de sortie
    cols = ["date", "permno", "prc"]
    if adjust == "split":
        cols.append("adjprc")
    return px[cols].sort_values(["date","permno"]).reset_index(drop=True)


# ------------------------------------------------------------
# 2) CRSP daily Market Cap = |PRC| * SHROUT * 1000
# ------------------------------------------------------------
def get_crsp_market_cap(db, permnos, start, end):
    """
    Market cap quotidienne (ME) en dollars : |PRC| * SHROUT * 1000
    PRC en $; SHROUT en milliers d’actions.
    """
    sql = f"""
        select date, permno, prc, shrout
        from crsp.dsf
        where permno in ({",".join(str(int(x)) for x in permnos)})
          and date between '{start}' and '{end}'
        order by date, permno
    """
    df = db.raw_sql(sql, date_cols=["date"])
    # to numeric + ME
    df["prc"] = pd.to_numeric(df["prc"], errors="coerce")
    df["shrout"] = pd.to_numeric(df["shrout"], errors="coerce")
    df["me"] = df["prc"].abs() * df["shrout"] * 1000.0  # dollars
    return df  # colonnes : date, permno, prc, shrout, me


# ------------------------------------------------------------
# 3) Fama-French daily factors (WRDS library 'ff')
# ------------------------------------------------------------
def get_ff6_daily(db, start, end, to_decimal=False):
    """
    Récupère FF5 + UMD (quotidien) sur WRDS et garantit les colonnes:
    date, mktrf, smb, hml, rmw, cma, rf, umd.
    - Si FF5 indisponible, on retombe sur FF3 et on met rmw/cma = NaN.
    - Si UMD indisponible, on crée umd = NaN.
    - On fait un merge outer pour garder l’union des dates (les manquants deviennent NaN).
    - Option: convertir de % vers décimaux (to_decimal=True).
    """
    import pandas as pd

    # 1) FF5 (fallback FF3 si besoin)
    ff5 = None
    for tbl in ["ff.five_factors_daily", "ff.five_factors_2x3_daily"]:
        try:
            ff5 = db.raw_sql(f"""
                select date, mktrf, smb, hml, rmw, cma, rf
                from {tbl}
                where date between '{start}' and '{end}'
                order by date
            """, date_cols=["date"])
            break
        except Exception:
            pass

    if ff5 is None:
        # Fallback FF3 -> on crée rmw & cma = NaN
        ff5 = db.raw_sql(f"""
            select date, mktrf, smb, hml, rf
            from ff.factors_daily
            where date between '{start}' and '{end}'
            order by date
        """, date_cols=["date"])
        ff5["rmw"] = pd.NA
        ff5["cma"] = pd.NA

    # Coercion numérique
    for col in ["mktrf", "smb", "hml", "rmw", "cma", "rf"]:
        ff5[col] = pd.to_numeric(ff5[col], errors="coerce")

    # 2) UMD (momentum) — fallback NaN si table indispo
    mom = None
    for tbl in ["ff.factors_momentum_daily", "ff.momentum_factors_daily"]:
        try:
            mom = db.raw_sql(f"""
                select date, umd
                from {tbl}
                where date between '{start}' and '{end}'
                order by date
            """, date_cols=["date"])
            break
        except Exception:
            pass

    if mom is None:
        # Pas d'accès UMD -> créer un DataFrame aligné sur les dates de ff5
        mom = ff5[["date"]].copy()
        mom["umd"] = pd.NA
    else:
        mom["umd"] = pd.to_numeric(mom["umd"], errors="coerce")

    # 3) Merge outer = union des dates -> les séries manquantes seront NaN
    ff = ff5.merge(mom, on="date", how="outer").sort_values("date")
    ff = ff.drop_duplicates(subset=["date"]).reset_index(drop=True)

    # 4) Option: passer de % à décimaux
    if to_decimal:
        cols = ["mktrf", "smb", "hml", "rmw", "cma", "rf", "umd"]
        ff[cols] = ff[cols] / 100.0

    return ff[["date", "mktrf", "smb", "hml", "rmw", "cma", "rf", "umd"]]



# ------------------------------------------------------------
# 3) get sector
# ------------------------------------------------------------
def get_crsp_sectors(db, permnos):
    """
    Récupère les codes SIC depuis CRSP et mappe vers les 12 secteurs Fama-French.
    Retourne un DataFrame avec permno, siccd, sector_name.
    """

    sql = f"""
        SELECT DISTINCT sn.permno, sn.siccd
        FROM crsp.stocknames sn
        WHERE sn.permno IN ({",".join(str(int(x)) for x in permnos)})
    """
    df = db.raw_sql(sql)

    # Coerce numeric SIC
    df["siccd"] = pd.to_numeric(df["siccd"], errors="coerce")

    # Mapping SIC → Fama-French 12 industries
    FF12_MAP = {
        "Consumer NonDurables": [(100, 999), (2000, 2399), (2700, 2749), (2770, 2799),
                                 (3100, 3199), (3940, 3989)],
        "Consumer Durables": [(2500, 2519), (2590, 2599), (3630, 3659), (3710, 3711),
                              (3714, 3714), (3716, 3716), (3750, 3751), (3792, 3792),
                              (3900, 3939), (3990, 3999)],
        "Manufacturing": [(2520, 2589), (2600, 2699), (2750, 2769), (3000, 3099),
                          (3200, 3569), (3580, 3629), (3700, 3709), (3712, 3713),
                          (3715, 3715), (3717, 3749), (3752, 3791), (3793, 3799)],
        "Energy": [(1200, 1399), (2900, 2999)],
        "Chemicals": [(2800, 2829), (2840, 2899)],
        "Business Equipment": [(3570, 3579), (3660, 3692), (3694, 3699),
                               (3810, 3829), (7370, 7379)],
        "Telecom": [(4800, 4899)],
        "Utilities": [(4900, 4949)],
        "Shops": [(5000, 5999)],
        "Health": [(2830, 2839), (3693, 3693), (3840, 3859), (8000, 8099)],
        "Money": [(6000, 6999)],
        "Other": [(0, 9999)],  # catch-all par défaut
    }

    def map_sic_to_ff12(sic):
        if pd.isna(sic):
            return "Other"
        sic = int(sic)
        for sector, ranges in FF12_MAP.items():
            for (low, high) in ranges:
                if low <= sic <= high:
                    return sector
        return "Other"

    df["sector_name"] = df["siccd"].apply(map_sic_to_ff12)

    return df[["permno", "siccd", "sector_name"]]


# ------------------------------------------------------------
# Run & export CSV
# ------------------------------------------------------------
if __name__ == "__main__":


    # 3) Fama-French daily (pas de pivot: pas par permno)
    ff_df = get_ff6_daily(db, START_DATE, END_DATE)  # to_decimal=True si tu veux en décimal
    ff_csv = os.path.join(OUTPUT_DIR, "returns_factor.csv")
    ff_df.to_csv(ff_csv, index=False)
    print(f"[OK] Facteurs FF sauvegardés -> {ff_csv}")

    # 1) RET (CRSP returns) -> pivot (date x permno)
    ret_df = get_crsp_returns(db, PERMNOS, START_DATE, END_DATE)
    ret_wide = ret_df.pivot_table(index="date", columns="permno", values="ret", aggfunc="first").reset_index()
    ret_csv = os.path.join(OUTPUT_DIR, "returns_stocks.csv")
    ret_wide.to_csv(ret_csv, index=False)
    print(f"[OK] RET sauvegardé -> {ret_csv}")

    # 2) PRIX (on prend adjprc si dispo, sinon prc) -> pivot
    price_df = get_crsp_prices(db, PERMNOS, START_DATE, END_DATE, adjust="split")
    val_col = "adjprc" if "adjprc" in price_df.columns else "prc"
    price_wide = price_df.pivot_table(index="date", columns="permno", values=val_col, aggfunc="first").reset_index()
    price_csv = os.path.join(OUTPUT_DIR, "stocks_price.csv")
    price_wide.to_csv(price_csv, index=False)
    print(f"[OK] Stock price -> {price_csv}")

    # 2bis) Market Cap -> pivot
    me_df = get_crsp_market_cap(db, PERMNOS, START_DATE, END_DATE)
    me_wide = me_df.pivot_table(index="date", columns="permno", values="me", aggfunc="first").reset_index()
    me_csv = os.path.join(OUTPUT_DIR, "market_cap.csv")
    me_wide.to_csv(me_csv, index=False)
    print(f"[OK] Market Cap sauvegardée -> {me_csv}")
    
    #secteur!
    df_secteurs = get_crsp_sectors(db, PERMNOS)
    sectors_csv = os.path.join(OUTPUT_DIR, "sector.csv")
    df_secteurs.to_csv(sectors_csv, index=False)
    print(f"[OK] Secteurs CRSP (FF12) sauvegardés -> {sectors_csv}")


    


[OK] Facteurs FF sauvegardés -> ./wrds_exports\returns_factor.csv
[OK] RET sauvegardé -> ./wrds_exports\returns_stocks.csv
[OK] Stock price -> ./wrds_exports\stocks_price.csv
[OK] Market Cap sauvegardée -> ./wrds_exports\market_cap.csv
[OK] Secteurs CRSP (FF12) sauvegardés -> ./wrds_exports\sector.csv
